Get some transactions
Add plaid merchant, add website

based on https://github.com/meetcleo/data-science/blob/master/transaction-enrichment/notebooks/transactions_categories/T1-24/Data_for_TM_annotations.ipynb



In [1]:
import pandas as pd
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from io import StringIO
from cleodata.utils.secrets import get_secret
from cleodata.sources.sync.sync import SyncDataSource
boto3.setup_default_session(profile_name='DataScientist-878877078763')
redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

Attempting to uninstrument while already uninstrumented


2024-05-31 13:47:00 [debug    ] fetching credentials          
2024-05-31 13:47:02 [info     ] Credentials acquired          
2024-05-31 13:47:02 [info     ] Built connection pool         


In [2]:
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import s3fs
from fastparquet import ParquetFile

In [5]:
def read_from_s3(path):
    """Read parquet files and combine them into a single dataframe"""
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")

    if len(all_paths_from_s3) > 0:
        s3 = s3fs.S3FileSystem()
        fp_obj = ParquetFile(
            all_paths_from_s3, open_with=s3.open
        )  # use s3fs as the filesystem
        data = fp_obj.to_pandas()
        return data
    elif len(all_paths_from_s3)==1:
        return pd.read_parquet(all_paths_from_s3[0])
    else:
        print(f"Nothing found")
        print(f"paths from a{all_paths_from_s3}")
    
def read_csv_s3(bucket, key):
    try:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'])
        return df
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print("Key doesn't match. Please check the key value entered.")


def list_s3_flies(base_path):
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
    return all_paths_from_s3

 - **Description 1**: Text description of transactions: Internal description or processed and cleared description from external provider 

 - **Description 2**: Text description of transactions, original from external provider. The text returned by the financial institution to describe the transaction. 

 - **Additional Merchant name (original):** Merchant name of transaction from external provider 

 - **Original Transfer counterparty name:** Extracted from some transaction information from external provider 

 - **Location:** If not empty, will include information as in following example:

{'lat': 28.607128, 'lon': -81.387253, 'city': 'Orlando', 'region': 'FL', 'address': '609 Lee Rd', 'country': None, 'postal_code': '32810', 'store_number': None}

- **Payment channel:** The channel used to make a payment. Possible values: online, in store, other: - online: transactions that took place online. - in store: transactions that were made at a physical location. - other: transactions that relate to banks, e.g. fees or deposits. 

- **Payee:** name of transfer receiver -+ Payer: name of transfer sender

- Google search link for “Additional Merchant name (original)”

- Google search link for “Description 1”

- Google search link for “Description 2”

- Google search link for “Original Transfer counterparty name”

- Google search link for “Description 2 and Original Transfer counterparty name”


In [6]:
#original query
# df = redshift_source.fetch_data( """
# with 
# presect_transactions as (
#     select 
#             tt.id,
#             tt.transaction_category_id,
#             tt.currency_code,
#             tt.amount,
#             tt.corrected_made_on, 
#             tt.made_on, 
#             tt.login_provider_additional_attributes,
#             tt.description,
#             tt.status
#     from transactions tt
#     where tt.made_on between '2024-01-20' and '2024-01-31'
#     limit 10000
# )

# select
#     tt.id,
#     tc.name as transaction_category_v1,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'detailed') 
#         as plaid_category_detailed,
#     m.name as merchant_name,
#     case when td.payee!='None' and td.payee is not null then td.payee
#         when td.payer!='None' and td.payer is not null then td.payer
#         else Null end as transfer_counterparty_name,

#     coalesce(tt.corrected_made_on, tt.made_on) as transaction_date,

#     case when tt.amount < 0 then 'Out'
#          when tt.amount > 0 then 'In'
#          else 'Zero'
#         end as direction, 
#     abs(tt.amount) as amount, 
#     tt.currency_code,
    
        
#     tt.description as description_1,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'original_description') 
#         as description_2,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'merchant_name') 
#         as additional_merchant_name,
#     td.company as original_transfer_counterparty_name,
    
#     td.payee,
#     td.payer,
    
#     json_extract_path_text(tt.login_provider_additional_attributes, 'payment_channel') 
#         as payment_channel,
#     json_extract_path_text(tt.login_provider_additional_attributes, 'location') 
#         as location,
#     tt.status

# from  presect_transactions tt
# left join transaction_categories tc on tc.id = tt.transaction_category_id
# left join transaction_descriptions td on td.transaction_id = tt.id
# left join merchants m on m.id = td.merchant_id and td.merchant_id is not null
# """)

In [7]:
sql_trans_with_merchant = """ 
with tt as (select
        tt.id,
        tt.amount,
        tt.corrected_made_on,
        tt.description,
        json_extract_path_text(tt.login_provider_additional_attributes, 'original_description')
        as original_description_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'merchant_name')
        as merchant_name_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'detailed')
        as category_detailed_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'primary')
        as category_primary_plaid,
        tt.made_on,
        tt.currency_code,

        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_channel')
        as payment_channel,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_method')
        as payment_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_processor_method')
        as payment_processor_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','city')
        as city,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','region')
        as region,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','country')
        as country,
        tt.transaction_category_id,
        tt.login_provider_additional_attributes,
        tt.status
    from transactions tt
    where tt.made_on between '2024-03-02' and '2024-03-03'
    limit 100 )

select  tt.id,
        tt.amount,
        coalesce(tt.corrected_made_on, tt.made_on) as transaction_date,
        tt.corrected_made_on,
        tt.description,
        json_extract_path_text(tt.login_provider_additional_attributes, 'original_description')
        as original_description_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'merchant_name')
        as merchant_name_plaid,
        mm.name as merchant_name,
        mm.website as merchant_website,
        mm.twitter_name,
        mm.latitude as merchant_latitude,
        mm.longitude as merchant_longitude,

        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'detailed')
        as category_detailed_plaid,
        json_extract_path_text(tt.login_provider_additional_attributes, 'personal_finance_category', 'primary')
        as category_primary_plaid,
        tt.made_on,
        tt.currency_code,

        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_channel')
        as payment_channel,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_method')
        as payment_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'payment_processor_method')
        as payment_processor_method,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','city')
        as city,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','region')
        as region,
        json_extract_path_text(tt.login_provider_additional_attributes, 'location','country')
        as country,
        tt.transaction_category_id,
        tt.login_provider_additional_attributes,
        tt.status,
        transaction_descriptions.merchant_id,
        case when transaction_descriptions.payee!='None' and transaction_descriptions.payee is not null then transaction_descriptions.payee
            when transaction_descriptions.payer!='None' and transaction_descriptions.payer is not null then transaction_descriptions.payer
            else Null end as transfer_counterparty_name
from tt
left join  transaction_descriptions on tt.id = transaction_descriptions.transaction_id
left join merchants mm on mm.id = transaction_descriptions.merchant_id

"""

In [8]:
start_date_s = '2024-05-13'
end_date_s = '2024-05-13'
date_range = pd.date_range(start=start_date_s, end=end_date_s)
# Convert the date range to a list of strings
date_list = date_range.strftime('%Y-%m-%d').tolist()
date_list

['2024-05-13']

In [9]:
def get_query(start_date_s,end_date_s):

  sql_fct_trans_with_merchant = f"""  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'original_description')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'merchant_name') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'transaction_type')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'name') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'type') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'website') AS counterparty_website,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'logo_url') AS counterparty_logo_url,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'entity_id') AS counterparty_entity_id,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'confidence_level') AS counterparty_confidence_level,
        ftt.merchant_id,
        ftt.company,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'personal_finance_category', 'detailed')
          as category_detailed_plaid,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'personal_finance_category', 'primary')
          as category_primary_plaid,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'payment_channel')
          as payment_channel,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'payment_method')
          as payment_method,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'payment_processor_method')
          as payment_processor_method,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'location','city')
          as city,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'location','region')
          as region,
          json_extract_path_text(ftt.login_provider_additional_attributes, 'location','country')
          as country,
          ftt.status,
        ftt.decline_code,
        ftt.login_provider_additional_attributes
  from analytics.fct_transactions ftt
  where corrected_made_on >= '{start_date_s}' and corrected_made_on <= '{end_date_s}'
  and original_description_plaid is not null
  and original_description_plaid != ''
  and original_description_plaid != ' '
  and ABS(amount) > 0.1
  --limit 5000000
  """
  return sql_fct_trans_with_merchant

In [10]:
print(get_query(start_date_s,end_date_s))

  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'original_description')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'merchant_name') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'transaction_type')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'name') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'type') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterparties', '0', 'website') AS counterparty_website,
      json_extract_path_text(ftt.login_provider_additional_attributes, 'counterp

In [11]:
run_one_off = 1

for one_date in date_list:
    start_date = one_date
    end_date = one_date
    if run_one_off==0 :
        df_trans_orig = redshift_source.fetch_data(sql_fct_trans_with_merchant)
    elif run_one_off == 1:
        dataset_id = datetime.now().strftime("%y%m%d_%H%M")
        sql_fct_trans_with_merchant = get_query(start_date, end_date)
        sql = f"""
        unload ('select * from ({sql_fct_trans_with_merchant.replace("'", "''")}
        )')
            to 's3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}'
            iam_role 'arn:aws:iam::878877078763:role/CleoRedshiftToS3'
            format as parquet
            cleanpath;"""
        print(sql)
        redshift_source.execute_query(sql)
        
        path_file = f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}"
        print(path_file)
        #df_trans = read_from_s3(path_file)

        # bucket_name = path_file.split('://')[1].split('/')[0]
        # key = path_file.split('://')[1].split(bucket_name)[1][1:]
        # print(f"bucket {bucket_name}")
        # print(f"key {key}")
        # df_trans  = read_csv_s3(bucket_name, key )
    elif run_one_off ==2:
        path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-20_2024-05-20"
        df_trans = read_from_s3(path_file)


        unload ('select * from (  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''original_description'')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, ''merchant_name'') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''transaction_type'')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''name'') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''type'') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''website'') AS counterparty_website,
      json_extract_path

In [12]:
path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-13_2024-05-13"

In [13]:
df_trans = read_from_s3(path_file)
df_trans.shape

In [ ]:
df_trans

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,category_primary_plaid,payment_channel,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes
0,9928286887,2024-05-13,-6.09,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,Pirate Ship,place,Pirate Ship,...,GENERAL_SERVICES,in store,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
1,9928286896,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,Insufficient Funds / Failed Transaction Fee,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
2,9928286897,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,Insufficient Funds / Failed Transaction Fee,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
3,9928286884,2024-05-13,-4.36,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,Pirate Ship,place,Pirate Ship,...,GENERAL_SERVICES,in store,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
4,9928286891,2024-05-13,-8.74,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,Pirate Ship,place,Pirate Ship,...,GENERAL_SERVICES,in store,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,9782665215,2024-05-13,-63.00,USD,ATM Withdrawal,ATM Withdrawal,Cash Withdrawal,,special,,...,TRANSFER_OUT,other,,,,,,None,,"{""datetime"": ""2024-05-13T18:54:25.000Z"", ""loca..."
4999996,9782665216,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,Cash Withdrawal,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca..."
4999997,9782665217,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,Cash Withdrawal,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca..."
4999998,9782665789,2024-05-13,9.75,USD,"Visa Money Transfer - Burroughs, Jerrod","Visa Money Transfer - Burroughs, Jerrod",None,,special,,...,TRANSFER_IN,other,,,,,,None,,"{""datetime"": ""2024-05-13T23:51:00.000Z"", ""loca..."


In [ ]:
path_file_processed = path_file.split('raw/')[0]+'processed/'+path_file.split('raw/')[1]
path_file_processed


's3://cleo-data-science/transaction_enrichment/experimental_data/caste/processed/trans_2024-05-13_2024-05-13'

In [ ]:
df_trans

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,category_primary_plaid,payment_channel,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes
0,9928286887,2024-05-13,-6.09,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,Pirate Ship,place,Pirate Ship,...,GENERAL_SERVICES,in store,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
1,9928286896,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,Insufficient Funds / Failed Transaction Fee,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
2,9928286897,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,Insufficient Funds / Failed Transaction Fee,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
3,9928286884,2024-05-13,-4.36,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,Pirate Ship,place,Pirate Ship,...,GENERAL_SERVICES,in store,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
4,9928286891,2024-05-13,-8.74,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,Pirate Ship,place,Pirate Ship,...,GENERAL_SERVICES,in store,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,9782665215,2024-05-13,-63.00,USD,ATM Withdrawal,ATM Withdrawal,Cash Withdrawal,,special,,...,TRANSFER_OUT,other,,,,,,None,,"{""datetime"": ""2024-05-13T18:54:25.000Z"", ""loca..."
4999996,9782665216,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,Cash Withdrawal,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca..."
4999997,9782665217,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,Cash Withdrawal,,special,,...,BANK_FEES,other,,,,,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca..."
4999998,9782665789,2024-05-13,9.75,USD,"Visa Money Transfer - Burroughs, Jerrod","Visa Money Transfer - Burroughs, Jerrod",None,,special,,...,TRANSFER_IN,other,,,,,,None,,"{""datetime"": ""2024-05-13T23:51:00.000Z"", ""loca..."


If merchant name is not present, user merchant_name_plaid. 
If merchant_name is not present, use counterparty_name



In [ ]:
import re
import numpy as np

In [ ]:
# Coalescing mercant names

#replace None with null
#replace Nan
# Replace empty spaces, None, and strings with only spaces with NaN
df_trans['merchant_name'] = df_trans['merchant_name'].replace(r'^\s*$', np.nan, regex=True)
df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].replace(r'^\s*$', np.nan, regex=True)


df_trans['merchant_name'] = df_trans['merchant_name'].map({'None':None})
df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].map({'':None,' ':None})
# create a column combined_merchant where we take the any merchant name : Cleo, or plaid, or counterparty 
df_trans['merchant_name_combined'] = df_trans['merchant_name'].combine_first(df_trans['merchant_name_plaid'])
df_trans['merchant_name_combined'] = df_trans['merchant_name_combined'].combine_first(df_trans['counterparty_name'])
# Remove data without merchant name for training data
df_trans_cln = df_trans[(~df_trans['merchant_name_combined'].isnull()) & ~df_trans['merchant_name_combined'].isin(['',' '])]

# Coalescing descriptions 
# if original_description_plaid is empty use description
df_trans['description_combined'] = df_trans['original_description_plaid'].combine_first(df_trans['description'])
df_trans['len_description'] = df_trans['description_combined'].apply(lambda x: len(x))
df_trans = df_trans[df_trans['len_description'] >2]
##
#replace 'other' with ''
df_trans['payment_channel_processed'] = df_trans['payment_channel'].apply(lambda x: 'None' if x == 'other' else x)
# do some light processing to make strings shorter
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid'].apply(lambda x: re.sub('\\\\+','\\\\',x))
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid_processed'].apply(lambda x: re.sub(r'\d{4,}', ' ', x))

# #if there are 4 or more integers replace with a space
# output_string = re.sub(r'\d{4,}', ' ', df_data_raw['sentence'][1000050])
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid_processed'].apply(lambda x: re.sub(' +',' ',x))
df_trans.reset_index(drop=True, inplace=True)

/var/folders/96/34ycxb693ss1n858z5ck1xwh0000gn/T/ipykernel_15570/2131297994.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['payment_channel_processed'] = df_trans['payment_channel'].apply(lambda x: 'None' if x == 'other' else x)
/var/folders/96/34ycxb693ss1n858z5ck1xwh0000gn/T/ipykernel_15570/2131297994.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid'].apply(lambda x: re.sub('\\\\+','\\\\',x))
/var/f

In [ ]:
# There are some cases where Chime is the merchant but it isn't mentioned in the description, so remove these
df_trans['Chime in descr'] = df_trans['original_description_plaid'].apply(lambda x: 'Chime' in x)

/var/folders/96/34ycxb693ss1n858z5ck1xwh0000gn/T/ipykernel_15570/1527262344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['Chime in descr'] = df_trans['original_description_plaid'].apply(lambda x: 'Chime' in x)


In [ ]:

df_trans = df_trans[(df_trans['Chime in descr'] & (df_trans['merchant_name_combined']=='Chime')) | (df_trans['merchant_name_combined']!='Chime')]
df_trans.reset_index(drop=True, inplace=True)
df_trans

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,description_combined,len_description,payment_channel_processed,original_description_plaid_processed,Chime in descr
0,9928286887,2024-05-13,-6.09,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,NaN,NaN,place,Pirate Ship,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Pirate Ship,External Withdrawal - Pirate Ship PLAID - 8444...,52,in store,External Withdrawal - Pirate Ship PLAID -,False
1,9928286896,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,NSF - Pirate Ship PLAID - 8444455854,36,None,NSF - Pirate Ship PLAID -,False
2,9928286897,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,NSF - Pirate Ship PLAID - 8444455854,36,None,NSF - Pirate Ship PLAID -,False
3,9928286884,2024-05-13,-4.36,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,NaN,NaN,place,Pirate Ship,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Pirate Ship,External Withdrawal - Pirate Ship PLAID - 8444...,52,in store,External Withdrawal - Pirate Ship PLAID -,False
4,9928286891,2024-05-13,-8.74,USD,External Withdrawal - Pirate Ship PLAID - 8444...,External Withdrawal - Pirate Ship PLAID - 8444...,NaN,NaN,place,Pirate Ship,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Pirate Ship,External Withdrawal - Pirate Ship PLAID - 8444...,52,in store,External Withdrawal - Pirate Ship PLAID -,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985960,9782665215,2024-05-13,-63.00,USD,ATM Withdrawal,ATM Withdrawal,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:25.000Z"", ""loca...",,ATM Withdrawal,14,None,ATM Withdrawal,False
4985961,9782665216,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca...",,Cash Withdrawal Fee,19,None,Cash Withdrawal Fee,False
4985962,9782665217,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca...",,Cash Withdrawal Fee,19,None,Cash Withdrawal Fee,False
4985963,9782665789,2024-05-13,9.75,USD,"Visa Money Transfer - Burroughs, Jerrod","Visa Money Transfer - Burroughs, Jerrod",NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T23:51:00.000Z"", ""loca...",,"Visa Money Transfer - Burroughs, Jerrod",39,None,"Visa Money Transfer - Burroughs, Jerrod",False


In [ ]:
df_trans_no_merchant = df_trans[(df_trans['merchant_name_combined'].isnull()) | df_trans['merchant_name_combined'].isin(['',' '])]

In [ ]:
df_trans_no_merchant

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,description_combined,len_description,payment_channel_processed,original_description_plaid_processed,Chime in descr
1,9928286896,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,NSF - Pirate Ship PLAID - 8444455854,36,None,NSF - Pirate Ship PLAID -,False
2,9928286897,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,NSF - Pirate Ship PLAID - 8444455854,36,None,NSF - Pirate Ship PLAID -,False
12,9928286894,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,NSF - Pirate Ship PLAID - 8444455854,36,None,NSF - Pirate Ship PLAID -,False
13,9928286895,2024-05-13,-10.00,USD,NSF - Pirate Ship PLAID - 8444455854,NSF - Pirate Ship PLAID - 8444455854,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,NSF - Pirate Ship PLAID - 8444455854,36,None,NSF - Pirate Ship PLAID -,False
17,9928365812,2024-05-13,1.50,USD,From Share 0000,From Share 0000,NaN,NaN,special,,...,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",,From Share 0000,15,None,From Share,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985959,9782665214,2024-05-13,-23.00,USD,ATM Withdrawal,ATM Withdrawal,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:25.000Z"", ""loca...",,ATM Withdrawal,14,None,ATM Withdrawal,False
4985960,9782665215,2024-05-13,-63.00,USD,ATM Withdrawal,ATM Withdrawal,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:25.000Z"", ""loca...",,ATM Withdrawal,14,None,ATM Withdrawal,False
4985961,9782665216,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca...",,Cash Withdrawal Fee,19,None,Cash Withdrawal Fee,False
4985962,9782665217,2024-05-13,-2.50,USD,Cash Withdrawal Fee,Cash Withdrawal Fee,NaN,NaN,special,,...,,None,,"{""datetime"": ""2024-05-13T18:54:29.000Z"", ""loca...",,Cash Withdrawal Fee,19,None,Cash Withdrawal Fee,False


In [ ]:
print(df_trans.shape)
print(df_trans_cln.shape)

(4985965, 34)
(4197208, 29)


In [ ]:
df_trans_cln['merchant_name_combined'].value_counts()

merchant_name_combined
Cash App                   346486
Zelle                      172242
Walmart                    115901
Apple                      114375
Earnin                     108338
                            ...  
Old River Donuts                1
Burnside                        1
Mjp Management                  1
Spotifyusai Angel Lopez         1
Anju House                      1
Name: count, Length: 426343, dtype: int64

In [ ]:
nunique_merchants = df_trans_cln['merchant_name_combined'].nunique()
print(f"There are {nunique_merchants}  in the dataset")

There are 360627  in the dataset


In [ ]:
df_counts = df_trans_cln['merchant_name_combined'].value_counts()
df_counts
df_counts[0:30]

merchant_name_combined
Cash App              282226
Zelle                 165804
Earnin                 99218
Walmart                90900
Apple                  88003
Amazon                 73613
Apple Cash             72555
McDonald''s            71907
Cleo                   71634
MoneyLion              59073
Dave                   55338
Uber                   49607
Venmo                  44853
Brigit                 41908
Empower                40598
Wells Fargo            39666
FanDuel Sportsbook     39414
AfterPay               37120
7-Eleven               35490
USAA                   35092
Albert                 33857
Shell                  33815
Klover App             32576
Starbucks              32561
Circle K               31527
PayPal                 31336
Dollar General         30757
Uber Eats              30436
Target                 29270
Zip.co                 28899
Name: count, dtype: int64

In [ ]:
nunique_merchants = df_trans_cln['merchant_name_combined'].nunique()
print(f"There are {nunique_merchants}  in the dataset")

There are 360627  in the dataset


In [ ]:
df_trans_cln[df_trans_cln['merchant_name_combined']=="Uber Eats"]

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_channel,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined
41,9796727574,2024-05-13,-46.58,USD,Uber Eats,PURCHASE AUTHORIZED ON 05/11 UBER EATS HELP.UB...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
1155,9935828615,2024-05-13,-36.97,USD,Uber Eats,Point of Sale Debit L340 DATE 05-11 UBER EATS ...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
1184,9796788873,2024-05-13,-21.45,USD,Uber Eats,PURCHASE 0513 UBER EATS HELP.UBER.COMCA XXXXX1...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
1185,9796788874,2024-05-13,-31.60,USD,Uber Eats,PURCHASE 0513 UBER EATS HELP.UBER.COMCA XXXXX1...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
1186,9796788875,2024-05-13,-41.27,USD,Uber Eats,PURCHASE 0511 UBER EATS HELP.UBER.COMCA XXXXX1...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999347,9796383979,2024-05-13,-96.51,USD,Uber Eats,POS Debit - Visa Check Card 9507 - UBER EATS H...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
4999348,9796383980,2024-05-13,-145.89,USD,Uber Eats,POS Debit - Visa Check Card 9507 - UBER EATS H...,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
4999527,9791989406,2024-05-13,-49.41,USD,Uber Eats,UBER EATS HELP.UBER.COM CA 05/13,NaN,NaN,place,Uber Eats,...,online,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats
4999685,9792002166,2024-05-13,-33.29,USD,Uber Eats,Pos Debit- 8123 8123 Uber Eats Help.uber....,NaN,NaN,place,Uber Eats,...,online,,,,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats


In [ ]:
print(df_trans_cln.shape)

(4197208, 29)


In [ ]:
# examples where the description and the merchant name are the same are probably not too informative
df_trans_cln3 = df_trans_cln[df_trans_cln['merchant_name_combined']!=df_trans_cln['original_description_plaid']]
print(df_trans_cln3.shape)
df_trans_cln3['merchant_name_combined'].value_counts()[:-40]

(4062826, 29)


merchant_name_combined
Cash App                    280486
Zelle                       165795
Earnin                       98186
Apple                        84493
Walmart                      83533
                             ...  
Elder''s Of Robbinsville         1
Feast Mwc                        1
City Smoke Stop                  1
Le Luxe                          1
Coffre-fort                      1
Name: count, Length: 345825, dtype: int64

In [ ]:
def look_up_for_one_merchant(merchant_name, df):
    return df[df['merchant_name_combined']==merchant_name]

In [ ]:
look_up_for_one_merchant('Affirm', df_trans_cln3)

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_channel,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined
825,9796774754,2024-05-13,-51.89,USD,AFFIRM INC AFFIRM PAY 240510 6318108 Gerardo *...,AFFIRM INC AFFIRM PAY 240510 6318108 Gerardo *...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
836,9796774767,2024-05-13,-10.96,USD,RECURRING PAYMENT AUTHORIZED ON 05/11 AFFIRM *...,RECURRING PAYMENT AUTHORIZED ON 05/11 AFFIRM *...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
1172,9796788863,2024-05-13,-10.00,USD,PURCHASE 0511 AFFIRM.COM PAYMENTS AFFIRM.COM C...,PURCHASE 0511 AFFIRM.COM PAYMENTS AFFIRM.COM C...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
1267,9805523323,2024-05-13,-21.07,USD,ACH Transaction - AFFIRM INC AFFIRM PAY 000210...,ACH Transaction - AFFIRM INC AFFIRM PAY 000210...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
2767,9796807924,2024-05-13,-36.73,USD,CHECKCARD 0512 AFFIRM.COM PAYMEN SAN FRANCISCO...,CHECKCARD 0512 AFFIRM.COM PAYMEN SAN FRANCISCO...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998741,9799166486,2024-05-13,-44.70,USD,POS Debit - Visa Check Card 7725 - AFFIRM PAY ...,POS Debit - Visa Check Card 7725 - AFFIRM PAY ...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
4998879,9799496971,2024-05-13,-16.58,USD,POS Debit - Visa Check Card 5853 - AFFIRM PAY ...,POS Debit - Visa Check Card 5853 - AFFIRM PAY ...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
4999213,9795867812,2024-05-13,-48.30,USD,POS Debit - Visa Check Card 9114 - AFFIRM COM ...,POS Debit - Visa Check Card 9114 - AFFIRM COM ...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm
4999936,9782662865,2024-05-13,-12.48,USD,AFFIRM INC TYPE: AFFIRM PAY ID: 7218 DATA: CO:...,AFFIRM INC TYPE: AFFIRM PAY ID: 7218 DATA: 85...,NaN,NaN,place,Affirm,...,other,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm


In [ ]:
df_trans_cln3.columns

Index(['transaction_id', 'corrected_made_on', 'amount', 'currency_code',
       'description', 'original_description_plaid', 'merchant_name',
       'merchant_name_plaid', 'transaction_type_plaid', 'counterparty_name',
       'counterparty_type', 'counterparty_website', 'counterparty_logo_url',
       'counterparty_entity_id', 'counterparty_confidence_level',
       'merchant_id', 'company', 'category_detailed_plaid',
       'category_primary_plaid', 'payment_channel', 'payment_method',
       'payment_processor_method', 'city', 'region', 'country', 'status',
       'decline_code', 'login_provider_additional_attributes',
       'merchant_name_combined'],
      dtype='object')

In [ ]:
columns_to_keep = ['transaction_id','corrected_made_on','amount','original_description_plaid','merchant_name_combined','payment_channel']

In [ ]:
df_trans_cln4 = df_trans_cln3.drop_duplicates(subset=['original_description_plaid','merchant_name_combined'])

In [ ]:
df_trans_cln4.shape

(3001739, 29)

In [ ]:
nunique_merchants = df_trans_cln4['merchant_name_combined'].nunique()
print(f"There are {nunique_merchants}  in the dataset")

In [ ]:
df_trans_cln4[columns_to_keep]

,transaction_id,corrected_made_on,amount,original_description_plaid,merchant_name_combined,payment_channel
0,9928286887,2024-05-13,-6.09,External Withdrawal - Pirate Ship PLAID - 8444...,Pirate Ship,in store
10,9928286898,2024-05-13,-69.95,POS Withdrawal - RAZZIS PIZZERIA 8523 GREENWOO...,Razzis Pizzeria,in store
16,9928350638,2024-05-13,-3.50,Withdrawal Debit Card CASH APP*KENNEDY BUCKD 8...,Cash App,other
19,9928417024,2024-05-13,19.70,APPLE CASH INS 05/12 #XXXXX3233 PMNT RCVD APPL...,Apple Cash,other
20,9928417026,2024-05-13,-5.12,MOBILE PURCHASE 0510 76 - DBA SKS PETROLEUM SE...,76 Gas Stations,online
...,...,...,...,...,...,...
4999974,9782663754,2024-05-13,-10.59,Amzn Mktp Us*P78 Ub1 Q93,Amazon,online
4999989,9782665052,2024-05-13,8.68,Payment VENMO*Cromidas Mike New York CityNYUS,Venmo,other
4999991,9782665049,2024-05-13,-7.99,Purchase SUPER SMOKE SHOP & MAR SALEM US,Super Smoke Shop & Mar,in store
4999992,9782665116,2024-05-13,-20.64,CVS/PHARMACY #06 06966--6,CVS,in store


In [ ]:
# save this dataset and depending on the type of algorithm you can choose how to reorder the data

df_trans_cln4.to_parquet(path_file_processed)

In [ ]:
print(f"Finsihed writing file {path_file_processed}")

In [ ]:
#df_trans_cln3.to_csv('/Users/claracastellanos/Documents/DATA/MERCHANTS/2024_05_20.csv')

In [ ]:
#load data back in to make sure everythin is ok
df_tmp = read_from_s3("s3://cleo-data-science/transaction_enrichment/experimental_data/caste/biencoder_test1/data1")

In [ ]:
df_tmp.shape

(956812, 30)

In [ ]:
df_tmp

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,Chime in descr
index,,,,,,,,,,,,,,,,,,,,,
0,9848923081,2024-05-20,25.00,USD,ZELLE P2P PAYMENT RECEIVED ON 05/19 \,ZELLE P2P PAYMENT RECEIVED ON 05/19 \\,None,None,special,Zelle,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Zelle,False
1,9848926035,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",None,None,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
2,9848926034,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",None,None,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
3,9848945525,2024-05-20,19.65,USD,Transfer from Cash App,Transfer from Cash App,None,None,special,Cash App,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Cash App,False
4,9848948208,2024-05-20,-100.00,USD,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,None,None,place,Q Nails,...,,,Newport,TN,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Q Nails,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170552,9857528533,2024-05-20,-63.27,USD,Cash App*Anthony Cenic,Cash App*Anthony Cenic,None,None,special,Cash App,...,,,,,,None,,"{""datetime"": ""2024-05-20T19:38:03.000Z"", ""loca...",Cash App,False
1170554,9857528559,2024-05-20,-34.70,USD,Point Of Sale Withdrawal 445488848992 DD DOORD...,Point Of Sale Withdrawal 445488848992 DD DOORD...,None,None,place,Mamichuyt,...,,,,CA,US,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Mamichuyt,False
1170555,9857528629,2024-05-20,-10.28,USD,Transfer to Chime Savings Account,Transfer to Chime Savings Account,None,None,special,Chime,...,,,,,,None,,"{""datetime"": ""2024-05-20T19:11:52.000Z"", ""loca...",Chime,True


In [ ]:
df_trans_cln3.to_parquet()